# Data exploration

## I.Import relevant librairires

In [ ]:
import os
import sys
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Below, you can import the different modules coded in ``WeakSegNet/src``

In [ ]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..")) # This is the repo directory: WeakSegNet/
SRC_PATH = os.path.join(BASE_DIR, "src")
sys.path.append(SRC_PATH)

from dataset import OxfordPet, data_loading, generate_mask, data_transform

## II.Load the data

In [ ]:
# Sanity check: are the trainval.txt and test.txt files available

FILE_PATH = os.path.join(BASE_DIR, "data")

split = "trainval" # or "test"
print(f"Looking for {split}.txt in:", os.path.join(FILE_PATH, "annotations", f"{split}.txt"))
print("Does the file exists?", os.path.exists(os.path.join(FILE_PATH, "annotations", "test.txt")))

## III.Data visualization

In [ ]:
# Data splits: configure the size of the splits
batch_size_train, batch_size_val, batch_size_test = 32, 16, 32
val_split = 0.2
size = (batch_size_train, batch_size_val, batch_size_test, val_split)

### 1.Fully supervised case

In [ ]:
# Configure which data you wish to load
fully_supervised = True
weakly_supervised = False
weakly_supervised_bbox_to_mask_dummy = False
experiment = (fully_supervised, weakly_supervised, weakly_supervised_bbox_to_mask_dummy)

In [ ]:
# Configure which data transformation you wish to load
image_size = 256
image_transform, mask_transform = data_transform()

In [ ]:
train_dataset_fs, val_dataset_fs, test_dataset_fs, train_loader_fs, val_loader_fs, test_loader_fs = data_loading(
                                                                    path=FILE_PATH,
                                                                    experiment=experiment,
                                                                    image_transform=image_transform,
                                                                    mask_transform=mask_transform,
                                                                    size=size
                                                                    )

In [ ]:
image_fs, mask_fs, info = train_dataset_fs[35]

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(image_fs.permute(1, 2, 0))
plt.title("Image")
plt.axis('off')
plt.grid(False)

plt.subplot(1, 2, 2)
plt.imshow(mask_fs.squeeze(), cmap="grey")
plt.title(f"Mask for image {info['name']}")
plt.axis('off')

plt.grid(False)
plt.show()

In [ ]:
unique_vals = torch.unique(mask_fs)
print("Unique mask values:", unique_vals)

### 2.Weakly supervised: bounding boxes

#### 2.1.Bounding boxes visualization

In [ ]:
# Configure which data you wish to load
fully_supervised = False
weakly_supervised = True
weakly_supervised_bbox_to_mask_dummy = False
experiment = (fully_supervised, weakly_supervised, weakly_supervised_bbox_to_mask_dummy)

In [ ]:
image_transform, mask_transform = None, None

In [ ]:
train_dataset_ws, val_dataset_ws, test_dataset_ws, train_loader_ws, val_loader_ws, test_loader_ws = data_loading(
                                                                    path=FILE_PATH,
                                                                    experiment=experiment,
                                                                    image_transform=image_transform,
                                                                    mask_transform=mask_transform,
                                                                    size=size
                                                                    )

In [ ]:
image_ws, info = train_dataset_ws[100]

In [ ]:
box = info['bbox']

In [ ]:
plt.imshow(image_ws)
plt.title("Image")
plt.axis('off')
plt.grid(False)
ax = plt.gca()
width = box[1] - box[0]
heigth = box[3] - box[2]
ax.add_patch(patches.Rectangle((box[0], box[2]), width, heigth, linewidth=2, edgecolor='r', facecolor='none'))
plt.show()

#### 2.2.Bounding boxes: dummy dataset

In [ ]:
# Configure which data you wish to load
fully_supervised = False
weakly_supervised = True
weakly_supervised_bbox_to_mask_dummy = True
experiment = (fully_supervised, weakly_supervised, weakly_supervised_bbox_to_mask_dummy)

In [ ]:
# Configure which data transformation you wish to load
image_size = 256
image_transform, mask_transform = data_transform(image_size=image_size)

In [ ]:
train_dataset_ws, val_dataset_ws, test_dataset_ws, train_loader_ws, val_loader_ws, test_loader_ws = data_loading(
                                                                    path=FILE_PATH,
                                                                    experiment=experiment,
                                                                    image_transform=image_transform,
                                                                    mask_transform=mask_transform,
                                                                    size=size
                                                                    )

In [ ]:
image_ws, weak_mask, info = val_dataset_ws[667]

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(image_ws.permute(1, 2, 0))
plt.title("Image")
plt.axis('off')
plt.grid(False)

plt.subplot(1, 2, 2)
plt.imshow(weak_mask.squeeze(), cmap="grey")
plt.title(f"Mask for image {info['name']}")
plt.axis('off')

plt.grid(False)
plt.show()

In [ ]:
unique_vals = torch.unique(weak_mask)
print("Unique mask values:", unique_vals)

### 3- Weakly supervised: CMAP